# Simulation of ARCS powder resolution function

In [ ]:
# some goodies
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy as np, histogram.hdf as hh, histogram as H

## Create workflow

** Change the workdir below !!! **

In [ ]:
workdir = "~/simulations/ARCS/powder-res-demo/"
!mkdir -p {workdir}
%cd {workdir}

In [ ]:
!mcvine workflow powder --instrument=ARCS --sample=V --workdir=res-sim

In [ ]:
!ls res-sim/

In [ ]:
simdir = '%s/res-sim' % workdir

## Beam simulation

In [ ]:
# change to beam sim directory
%cd {simdir}/beam

In [ ]:
# show the simulation script
!cat run-beam.sh

In [ ]:
# modify the simulation script (desired incident energy, neutron count)

In [ ]:
%%file run-beam.sh
#!/usr/bin/env bash

mcvine instruments arcs beam --keep-in-cache --use-cache -E=300 --ncount=1e8 --nodes=10

In [ ]:
# show the simulation script again to double check
!cat run-beam.sh

In [ ]:
# if clean up is necessary, uncomment the following line
# rm -rf *out *.params arcs_moderator2sample.pml *.dat run-m2s.sh

In [ ]:
%%time
%%script bash --out out_beam --err err_beam
# run simulation
./run-beam.sh

After the beam simulation is done
* The main output are in the new folder "out".
* The ".h5" files are histograms of various sorts.
* The "neutron" file is the simulated neutrons of the beam incident on the sample

In [ ]:
!ls out

In [ ]:
# plot I(E) spectrum
ie = hh.load("out/ienergy.h5")
plt.plot(ie.energy, ie.I)

In [ ]:
# plot monitor 1 I(tof) spectrum
m1 = hh.load("out/mon1-itof-focused.h5")
plt.plot(m1.tof, m1.I)

## Scattering simulation and reduction

The simulation directory contains a Makefile and various scripts to make it easy to run the simulation and analsysis.

In [ ]:
%cd {simdir}
!ls

### Change ncount and nodes for the scattering simulation

In [ ]:
# scatter is the script for sample scattering simulation
!cat scatter

In [ ]:
%%file scatter
#!/usr/bin/env bash

LAUNCHER=`python -c "from mcni.pyre_support.MpiApplication import mpi_launcher_choice as l; print l"`
rm -rf out && ./sss --ncount=1e6 --multiple-scattering=off --$LAUNCHER.nodes=10 --buffer_size=10000

### Also change to script that creates the simulated nexus file

In [ ]:
%%file create-nxs
#!/usr/bin/env bash

rm -rf work-arcs-neutrons2nxs
time mcvine instruments arcs neutrons2nxs --neutrons=out/scattered-neutrons --nodes=10

### Change sample kernel
Set the momentum-transfer and energy-transfer

In [ ]:
%%file sampleassembly/V-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- weights: absorption, scattering, transmission -->
<homogeneous_scatterer mcweights="0, 1, 0">
  
  <ConstantQEKernel momentum-transfer="7/angstrom" energy-transfer="0*meV">
  </ConstantQEKernel>
  
</homogeneous_scatterer>

### Run Simulation

In [ ]:
%%time
%%script bash --out out_scatter --err err_scatter
# run simulation and reduction
chmod +x scatter sss reduce2iqe create-nxs
make

## Inspect

In [ ]:
iqe = hh.load("iqe.h5")
H.plot(iqe, min=0, max=1e-4)

In [ ]:
# Get energy spectrum
# Change the range below
# (Qmin, Qmax), (Emin, Emax)
ie = iqe[(6,8), (-100, 50)].sum('Q')

In [ ]:
H.plot(ie)